In [1]:
!pip install comet_ml cartopy h5netcdf netcdf4 scipy einops torchdiffeq torchsummary

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset,ConcatDataset
from tqdm import tqdm
import os
import xarray as xr
from dataset import *
from train import Trainer  
from PIL import Image
from comet_ml import Experiment
import numpy as np
import pandas as pd
from loguru import logger
import sys
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
sys.path.append('/home/jovyan/work/spatial-temporal/models/graph_grid_model')
logger.add("./logs/training_log.log", rotation="1 MB", level="INFO")

[4pdvGPU Msg(3794:140099667196800:libvgpu.c:869)]: Initializing.....
[4pdvGPU Warn(3794:140099667196800:hook.c:475)]: remap handles for device 0
[4pdvGPU Warn(3794:140099667196800:multiprocess_memory_limit.c:568)]: Kick dead proc 3512


1

In [3]:
time_slice_train = ("2014-01-01", "2017-12-31")  # 加载指定时间范围的数据
time_slice_val = ('2018-01-01', '2018-6-30') # 加载指定时间范围的数据
time_slice_test = ('2018-07-01', '2018-12-31')  # 加载指定时间范围的数据
# time_slice_train = ("2014-01-01", "2014-01-10")  # 加载指定时间范围的数据
# time_slice_val = ("2014-01-11", "2014-01-17") # 加载指定时间范围的数据
# time_slice_test = ("2014-01-14", "2014-01-22")  # 加载指定时间范围的数据
# variables for grid  weatherbench
grid_root_dir = '/home/jovyan/work/data/all_1.40625deg/2m_temperature/media/rasp/Elements/weather-benchmark/1.40625deg'
grid_variables = ["2m_temperature"]  # 如果为 None 则加载所有变量
grid_step = 1  # 每隔 6 个时间步加载一个数据
grid_time_window = 6
grid_pred_window = 6


# variables for graph  weather-5k
# graph_root_dir = '/home/jovyan/work/data/WEATHER-5K/global_weather_stations'
graph_root_dir = '/home/jovyan/work/data/WEATHER-5K/global_weather_stations_test'
graph_variables = ['TMP']
graph_step = 6
graph_time_window = 6
graph_pred_window = 6


grid_dataset_train = WeatherBenchDataset(grid_root_dir,
                                   input_variables=grid_variables,
                                   target_variables=grid_variables,
                                   time_window = grid_time_window,
                                   pred_window=grid_pred_window,
                                   time_slice=time_slice_train,
                                   step=grid_step,
                                   transform=None,
                                   )
grid_dataset_val = WeatherBenchDataset(grid_root_dir,
                                   input_variables=grid_variables,
                                   target_variables=grid_variables,
                                   time_window = grid_time_window,
                                   pred_window=grid_pred_window,
                                   time_slice=time_slice_val,
                                   step=grid_step,
                                   transform=None,)
grid_dataset_test = WeatherBenchDataset(grid_root_dir,
                                   input_variables=grid_variables,
                                   target_variables=grid_variables,
                                   time_window = grid_time_window,
                                   pred_window=grid_pred_window,
                                   time_slice=time_slice_test,
                                   step=grid_step,
                                   transform=None,)


graph_dataset_train = Weather5kDataset(root_dir=graph_root_dir, 
                                 variables=graph_variables, 
                                 time_window=graph_time_window, 
                                 pred_window=graph_pred_window, 
                                 time_slice=time_slice_train, 
                                 step=graph_step,
                                 target_variables=['TMP'])
graph_dataset_val = Weather5kDataset(root_dir=graph_root_dir, 
                                 variables=graph_variables, 
                                 time_window=graph_time_window, 
                                 pred_window=graph_pred_window, 
                                 time_slice=time_slice_val, 
                                 step=graph_step,
                                 target_variables=['TMP'])
graph_dataset_test = Weather5kDataset(root_dir=graph_root_dir, 
                                 variables=graph_variables, 
                                 time_window=graph_time_window, 
                                 pred_window=graph_pred_window, 
                                 time_slice=time_slice_test, 
                                 step=graph_step,
                                 target_variables=['TMP'])


Loaded 35064 time steps for input variables ['2m_temperature'] and target variables ['2m_temperature'] with step 1. Total input channels: 1, Total target channels: 1.
Loaded 4344 time steps for input variables ['2m_temperature'] and target variables ['2m_temperature'] with step 1. Total input channels: 1, Total target channels: 1.
Loaded 4416 time steps for input variables ['2m_temperature'] and target variables ['2m_temperature'] with step 1. Total input channels: 1, Total target channels: 1.
Generating timestamps...
Timestamps generated: 35041 points.
Loading and filtering station data...


100%|██████████| 351/351 [00:30<00:00, 11.50it/s]


Data loaded successfully.
Generating timestamps...
Timestamps generated: 4321 points.
Loading and filtering station data...


100%|██████████| 351/351 [00:28<00:00, 12.28it/s]


Data loaded successfully.
Generating timestamps...
Timestamps generated: 4393 points.
Loading and filtering station data...


100%|██████████| 351/351 [00:28<00:00, 12.42it/s]

Data loaded successfully.


In [4]:
print(grid_dataset_train.__len__())
y = grid_dataset_train[1]
for j in y:
    print(j.shape)
    # print(j)
    break

x = graph_dataset_train[1]
for i in x:
    print(i.shape)
    # print(i)
    break

5830
torch.Size([6, 1, 128, 256])
torch.Size([6, 351, 1])


In [5]:
graph_grid_dataset_train = GridGraphDataset(grid_dataset_train, graph_dataset_train, grid_step, graph_step)
graph_grid_dataset_val = GridGraphDataset(grid_dataset_val, graph_dataset_val, grid_step, graph_step)
graph_grid_dataset_test = GridGraphDataset(grid_dataset_test, graph_dataset_test, grid_step, graph_step)
# graph_grid_dataset_train = GridGraphDataset(grid_dataset_train, graph_dataset_train)
# graph_grid_dataset_val = GridGraphDataset(grid_dataset_val, graph_dataset_val)
# graph_grid_dataset_test = GridGraphDataset(grid_dataset_test, graph_dataset_test)
batch_size = 12  # 假设批次大小为 2

train_loader = DataLoader(graph_grid_dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(graph_grid_dataset_val, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(graph_grid_dataset_test, batch_size=batch_size, shuffle=False, num_workers=0)

for i, (input_tensor_grid, input_tensor_graph, output_tensor_grid, output_tensor_graph, time_grid, time_graph, lan_lon) in enumerate(test_loader):
    # print(input_tensor_grid.shape)
    # print("max: ", torch.max(lan_lon))
    # print("min: ", torch.min(lan_lon))
    has_nan = torch.isnan(input_tensor_graph).any()
    # print(has_nan)
    print(lan_lon)
    
    print(input_tensor_graph.shape)
    print(output_tensor_graph.shape)
    print(input_tensor_grid.shape)
    print(output_tensor_grid.shape)
    print(time_grid.shape)
    print(time_graph.shape)
    print(lan_lon.shape)
    break
    

tensor([[[0.8731, 0.5811],
         [0.8850, 0.5448],
         [0.8893, 0.5829],
         ...,
         [0.7833, 0.5141],
         [0.7789, 0.5146],
         [0.7780, 0.5150]],

        [[0.8731, 0.5811],
         [0.8850, 0.5448],
         [0.8893, 0.5829],
         ...,
         [0.7833, 0.5141],
         [0.7789, 0.5146],
         [0.7780, 0.5150]],

        [[0.8731, 0.5811],
         [0.8850, 0.5448],
         [0.8893, 0.5829],
         ...,
         [0.7833, 0.5141],
         [0.7789, 0.5146],
         [0.7780, 0.5150]],

        ...,

        [[0.8731, 0.5811],
         [0.8850, 0.5448],
         [0.8893, 0.5829],
         ...,
         [0.7833, 0.5141],
         [0.7789, 0.5146],
         [0.7780, 0.5150]],

        [[0.8731, 0.5811],
         [0.8850, 0.5448],
         [0.8893, 0.5829],
         ...,
         [0.7833, 0.5141],
         [0.7789, 0.5146],
         [0.7780, 0.5150]],

        [[0.8731, 0.5811],
         [0.8850, 0.5448],
         [0.8893, 0.5829],
         ...,
 

In [6]:
output_model_path = '/home/jovyan/work/spatial-temporal/outputs/0401/models'
output_predict_path = '/home/jovyan/work/spatial-temporal/outputs/0401/preds'

h, w = 128, 256  # Grid height and width
T, T_prime = 6, 6  # Number of time steps for grid and graph
C, C_prime = 1, 1  # Number of channels for grid and graph
N = 5671
feature_dims = 32  # Node feature dimensionality
target_t = 6
target_v = 1

input_dim = 1  # Input dimension for time embeddings
patch_size = 16  # Patch size for grid2graph module


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(output_model_path):
    os.mkdir(output_model_path)
    print(f"create folder {output_model_path}")
    
if not os.path.exists(output_predict_path):
    os.mkdir(output_predict_path)
    print(f"create folder {output_predict_path}")

[4pdvGPU Warn(3794:140099667196800:utils.c:228)]: get default cuda 1 from (null)
[4pdvGPU Msg(3794:140099667196800:libvgpu.c:902)]: Initialized


In [7]:
from models.graph_grid_model.stg2cm import *

h, w = 128, 256  # Grid height and width
T, T_prime = 6, 6  # Number of time steps for grid and graph
C, C_prime = 1, 1  # Number of channels for grid and graph
N = 5671  # Number of graph nodes
feature_dims = 32  # Node feature dimensionality
input_dim = 1  # Input dimension for time embeddings
patch_size = 16  # Patch size for grid2graph module
num_epochs = 50
learning_rate = 2e-5 
target_t_grid = 6
target_t_graph = 6
target_v = 1


model = STG2CM(h,
            w,
            patch_size,
            grid_input_dim = C,
            grid_feature_dim = feature_dims,
            grid_input_t = T,

            graph_nodes=N,
            graph_input_dim = C_prime,
            graph_feature_dim = feature_dims,
            graph_input_t = T_prime,

            target_dim_v = target_v,
            target_dim_t_grid = target_t_grid,
            target_dim_t_graph = target_t_graph)

model.to(device)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = CyclicLR(optimizer, base_lr=learning_rate, max_lr=1e-3, cycle_momentum=False)

In [8]:
experiment = Experiment(
    api_key="xxx",  # 替换为你的 Comet API Key
    project_name="spatialtemporal",
    workspace="yyy"  # 替换为你的 Comet 工作区
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/yyy/spatialtemporal/177ac9f7a3094deb89379f031addeee4



In [9]:
##### Comet.ml 实验记录
experiment.log_parameters({
    "batch_size": batch_size,
    "num_epochs": num_epochs,
    "learning_rate": learning_rate
})

# 训练器实例化
trainer = Trainer(
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    output_path=output_model_path,
    experiment=experiment
)

# 训练循环
for epoch in range(num_epochs):
    logger.info(f"Epoch [{epoch+1}/{num_epochs}]")
    train_loss = trainer.train_one_epoch_STG2CM()

    val_loss, val_metrics = trainer.validate()

# 测试
test_loss, test_metrics = trainer.test()


# 结束实验
experiment.end()


2025-04-08 08:45:56.544 | INFO     | __main__:<module>:23 - Epoch [1/50]
Training:   0%|          | 0/486 [00:00<?, ?it/s]

grid_data.shape: torch.Size([12, 6, 1, 128, 256])
ode x.shape: torch.Size([12, 16, 6, 128, 256])


Training:   0%|          | 0/486 [00:22<?, ?it/s]

ode sol.shape: torch.Size([6, 12, 16, 6, 128, 256])
ode sol_out.shape: torch.Size([12, 6, 1, 128, 256])
grid_data_ode.shape: torch.Size([12, 6, 128, 256, 1])
combined_grid.shape: torch.Size([12, 6, 32, 128, 256])
graph_attn_output shape  torch.Size([12, 6, 32])


EinopsError:  Error while processing rearrange-reduction pattern "b (t n) c -> b t n c".
 Input tensor shape: torch.Size([12, 6, 32]). Additional info: {'t': 6, 'n': 351}.
 Shape mismatch, 6 != 2106

In [ ]:
print(graph_dataset_train.__len__())
print(grid_dataset_train.__len__())
len(graph_dataset_train.timestamps)

In [ ]:
def test_model(model, test_loader, device, save_dir="test_outputs"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    save_path = os.path.join(save_dir, f"test_{timestamp}")
    
    # 创建独立目录
    grid_dir = os.path.join(save_path, "grid")
    graph_dir = os.path.join(save_path, "graph")
    os.makedirs(grid_dir, exist_ok=True)
    os.makedirs(graph_dir, exist_ok=True)
    
    metrics = {"grid_mse": 0.0, "grid_rmse": 0.0, "graph_mse": 0.0, "graph_rmse": 0.0, "total_samples": 0}
    
    # 标记是否已保存 lan_lon
    lan_lon_saved = False
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (input_grid, input_graph, target_grid, target_graph, time_grid, time_graph, lan_lon) in enumerate(tqdm(test_loader)):
            # 确保所有输入张量在相同设备上
            input_grid = input_grid.to(device)
            input_graph = input_graph.to(device)
            target_grid = target_grid.to(device)
            target_graph = target_graph.to(device)
            time_grid = time_grid.to(device)      # 时间相关张量转移到设备
            time_graph = time_graph.to(device)    # 时间相关张量转移到设备
            lan_lon = lan_lon.to(device)          # 经纬度转移到设备
            
            # 仅保存一次 lan_lon
            if not lan_lon_saved:
                np.save(os.path.join(save_path, "lan_lon.npy"), lan_lon.cpu().numpy())
                lan_lon_saved = True
            
            # 模型推理
            pred_grid, pred_graph = model(
                input_graph,   # 确保输入顺序与模型定义一致
                input_grid,
                lan_lon,
                time_graph,
                time_grid
            )
            
            # 保存Grid和Graph数据
            np.save(os.path.join(grid_dir, f"pred_batch_{batch_idx}.npy"), pred_grid.cpu().numpy())
            np.save(os.path.join(grid_dir, f"target_batch_{batch_idx}.npy"), target_grid.cpu().numpy())
            np.save(os.path.join(graph_dir, f"pred_batch_{batch_idx}.npy"), pred_graph.cpu().numpy())
            np.save(os.path.join(graph_dir, f"target_batch_{batch_idx}.npy"), target_graph.cpu().numpy())
            
            # 计算指标
            batch_size = input_grid.size(0)
            metrics["total_samples"] += batch_size
            metrics["grid_mse"] += torch.mean((pred_grid - target_grid)**2).item() * batch_size
            metrics["graph_mse"] += torch.mean((pred_graph - target_graph)**2).item() * batch_size
    
    # 计算RMSE
    metrics["grid_rmse"] = np.sqrt(metrics["grid_mse"] / metrics["total_samples"])
    metrics["graph_rmse"] = np.sqrt(metrics["graph_mse"] / metrics["total_samples"])
    metrics["grid_mse"] /= metrics["total_samples"]
    metrics["graph_mse"] /= metrics["total_samples"]
    
    # 保存指标
    with open(os.path.join(save_path, "metrics.txt"), "w") as f:
        f.write(f"Grid MSE: {metrics['grid_mse']:.4f}\nGrid RMSE: {metrics['grid_rmse']:.4f}\n")
        f.write(f"Graph MSE: {metrics['graph_mse']:.4f}\nGraph RMSE: {metrics['graph_rmse']:.4f}\n")
    
    print(f"结果保存至: {save_path}")
    return metrics


best_model_path = os.path.join(output_model_path, 'model_epoch=1_RMSE=0.0176.pth')
model.load_state_dict(torch.load(best_model_path, map_location=device))
logger.info(f"Loaded best model from {best_model_path}")
test_metrics = test_model(model, val_loader, device, save_dir=output_predict_path)
print("\n最终测试指标:")
print(f"Grid MSE: {test_metrics['grid_mse']:.4f}")
print(f"Grid RMSE: {test_metrics['grid_rmse']:.4f}")
print(f"Graph MSE: {test_metrics['graph_mse']:.4f}")
print(f"Graph RMSE: {test_metrics['graph_rmse']:.4f}")

In [ ]:
##### import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
from tqdm import tqdm

def visualize_grid(grid_dir, batch_id=0, step=1, save_path="grid_animation.gif"):
    """
    生成Grid预测与标签的对比动态图
    Args:
        grid_dir: Grid数据目录路径
        batch_id: 要可视化的Batch编号
        step: 时间步间隔
        save_path: GIF保存路径
    """
    # 加载数据
    pred_path = os.path.join(grid_dir, f"pred_batch_{batch_id}.npy")
    target_path = os.path.join(grid_dir, f"target_batch_{batch_id}.npy")
    pred = np.load(pred_path)  # 形状应为 (B, T, C, H, W)
    target = np.load(target_path)
    
    # 检查数据维度
    if pred.ndim != 5:
        raise ValueError(f"Grid预测数据维度应为 (B, T, C, H, W)，当前维度为 {pred.shape}")
    
    # 取第一个样本和第一个通道
    sample_idx = 0
    channel_idx = 0
    pred = pred[sample_idx, :, channel_idx]  # (T, H, W)
    target = target[sample_idx, :, channel_idx]

    # 生成帧
    frames = []
    for t in range(0, pred.shape[0], step):  # 正确遍历时间步维度 T
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # 预测结果
        im1 = axes[0].imshow(pred[t], cmap="viridis", vmin=target.min(), vmax=target.max())
        axes[0].set_title("Prediction")
        plt.colorbar(im1, ax=axes[0])
        
        # 真实标签
        im2 = axes[1].imshow(target[t], cmap="viridis", vmin=target.min(), vmax=target.max())
        axes[1].set_title("Ground Truth")
        plt.colorbar(im2, ax=axes[1])
        
        # 计算指标
        mse = np.mean((pred[t] - target[t]) ** 2)
        rmse = np.sqrt(mse)
        
        # 添加标题
        plt.suptitle(f"Grid Predictions vs Targets (t={t})\nMSE: {mse:.4f}, RMSE: {rmse:.4f}", fontsize=14)
        plt.tight_layout()
        
        # 转为图像帧
        fig.canvas.draw()
        frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(frame)
        plt.close()
    
    # 保存GIF
    imageio.mimsave(save_path, frames, duration=500)
    print(f"Grid动态图已保存至: {save_path} (共{len(frames)}帧)")
    
def visualize_graph(graph_dir, lan_lon_path, batch_id=0, step=1, save_path="graph_animation.gif"):
    """
    生成Graph预测与标签的对比动态图
    Args:
        graph_dir: Graph数据目录路径
        lan_lon_path: 经纬度文件路径
        batch_id: 要可视化的Batch编号
        step: 时间步间隔
        save_path: GIF保存路径
    """
    # 加载数据
    pred_path = os.path.join(graph_dir, f"pred_batch_{batch_id}.npy")
    target_path = os.path.join(graph_dir, f"target_batch_{batch_id}.npy")
    pred = np.load(pred_path)  # 形状应为 (B, T, N, C)
    target = np.load(target_path)
    lan_lon = np.load(lan_lon_path)  # 形状应为 (B, N, 2)
    
    # 检查数据维度
    if pred.ndim != 4:
        raise ValueError(f"Graph预测数据维度应为 (B, T, N, C)，当前维度为 {55555pred.shape}")
    
    # 取第一个样本和第一个通道
    sample_idx = 0
    channel_idx = 0
    pred = pred[sample_idx, :, :, channel_idx]  # (T, N)
    target = target[sample_idx, :, :, channel_idx]
    coordinates = lan_lon[sample_idx]  # (N, 2)

    # 生成帧
    frames = []
    for t in range(0, pred.shape[0], step):  # 正确遍历时间步维度 T
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # 预测结果
        sc1 = axes[0].scatter(
            coordinates[:, 0], coordinates[:, 1], 
            c=pred[t], cmap="coolwarm", s=50, 
            vmin=target.min(), vmax=target.max()
        )
        axes[0].set_title("Prediction")
        plt.colorbar(sc1, ax=axes[0])
        
        # 真实标签
        sc2 = axes[1].scatter(
            coordinates[:, 0], coordinates[:, 1], 
            c=target[t], cmap="coolwarm", s=50, 
            vmin=target.min(), vmax=target.max()
        )
        axes[1].set_title("Ground Truth")
        plt.colorbar(sc2, ax=axes[1])
        
        # 计算指标
        mse = np.mean((pred[t] - target[t]) ** 2)
        rmse = np.sqrt(mse)
        
        # 添加标题
        plt.suptitle(f"Graph Predictions vs Targets (t={t})\nMSE: {mse:.4f}, RMSE: {rmse:.4f}", fontsize=14)
        plt.tight_layout()
        
        # 转为图像帧
        fig.canvas.draw()
        frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
        frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        frames.append(frame)
        plt.close()
    
    # 保存GIF
    imageio.mimsave(save_path, frames, duration=500)
    print(f"Graph动态图已保存至: {save_path} (共{len(frames)}帧)")
# 1. 加载测试结果（根据你的保存路径调整）
# test_output_dir = "/home/jovyan/work/spatial-temporal/outputs/0126/preds/test_20250325_154036"
test_dir = "/home/jovyan/work/spatial-temporal/outputs/0126/preds/test_20250325_162634"

# 生成Grid动态图（第0个Batch）
visualize_grid(
    grid_dir=os.path.join(test_dir, "grid"),
    batch_id=0,
    step=2,
    save_path=os.path.join(test_dir, "grid_animation.gif")
)

# 生成Graph动态图（第0个Batch）
visualize_graph(
    graph_dir=os.path.join(test_dir, "graph"),
    lan_lon_path=os.path.join(test_dir, "lan_lon.npy"),
    batch_id=0,
    step=2,
    save_path=os.path.join(test_dir, "graph_animation.gif")
)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from PIL import Image
from tqdm import tqdm
import xarray as xr
def calculate_errors(pred, label):
    # 均方误差 (Mean Squared Error)
    mse = np.mean((pred - label) ** 2)
    
    # 均方根误差 (Root Mean Squared Error)
    rmse = np.sqrt(mse)
    
    # 平均绝对误差 (Mean Absolute Error)
    mae = np.mean(np.abs(pred - label))
    
    # 区域平均误差 (Zonal Mean Error)
    zonal_mean_error = np.mean(np.abs(pred - label), axis=1).mean()
    
    # 技能评分 (Anomaly Correlation Coefficient, ACC)
    mean_pred = np.mean(pred)
    mean_label = np.mean(label)
    cov_pred_label = np.mean((pred - mean_pred) * (label - mean_label))
    std_pred = np.std(pred)
    std_label = np.std(label)
    
    if std_pred > 0 and std_label > 0:
        skill_score = cov_pred_label / (std_pred * std_label)
    else:
        skill_score = None
    
    return {
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "Skill Score": skill_score,
        "Zonal Mean Error": zonal_mean_error
    }

# 加载预测结果和真实标签
prediction_file = os.path.join(output_predict_path, 'predictions.nc')
label_file = os.path.join(output_predict_path, 'labels.nc')

predictions = xr.open_dataset(prediction_file)
label_data = xr.open_dataset(label_file)

# 检查数据集结构
print("Predictions dataset:")
print(predictions)
print("\nLabels dataset:")
print(label_data)

# 确保有时间坐标
if 'time' not in predictions.coords or 'time' not in label_data.coords:
    raise KeyError("Datasets do not contain 'time' coordinates.")

# 假设 x 和 y 坐标不存在，则生成默认的网格坐标
height, width = predictions["prediction"].shape[-2:]
x_coords = np.linspace(-180, 180, width)  # 示例经度范围
y_coords = np.linspace(-90, 90, height)   # 示例纬度范围

gif_filename = os.path.join(output_predict_path, "predictions_with_errors.gif")

step = 100  # 设置步长以减少帧数
print(f"Total frames: {len(predictions['time'])}, Displaying every {step} frames.")

frames = []
for i in tqdm(range(0, len(predictions["time"]), step), desc="Generating frames"):
    # 获取当前时间步的预测值和真实值
    pred_t2m = predictions["prediction"].isel(time=i).values.squeeze()
    label_t2m = label_data["label"].isel(time=i).values.squeeze()

    # 计算误差指标
    errors = calculate_errors(pred_t2m, label_t2m)

    # 绘图
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    fig.suptitle(f"Time: {predictions['time'][i].values}\n" +
                 f"MSE: {errors['MSE']:.4f}, RMSE: {errors['RMSE']:.4f}, MAE: {errors['MAE']:.4f}, " +
                 (f"Skill Score: {errors['Skill Score']:.4f}" if errors['Skill Score'] is not None else "") +
                 f", Zonal Mean Error: {errors['Zonal Mean Error']:.4f}")

    # 显示预测结果
    ax[0].set_title("Prediction")
    im = ax[0].pcolormesh(x_coords, y_coords, pred_t2m, shading="auto", cmap="coolwarm")
    plt.colorbar(im, ax=ax[0], label="Temperature (K)")
    ax[0].set_xlabel("Longitude")
    ax[0].set_ylabel("Latitude")

    # 显示真实值
    ax[1].set_title("Ground Truth")
    im = ax[1].pcolormesh(x_coords, y_coords, label_t2m, shading="auto", cmap="coolwarm")
    plt.colorbar(im, ax=ax[1], label="Temperature (K)")
    ax[1].set_xlabel("Longitude")
    ax[1].set_ylabel("Latitude")

    # 将 matplotlib Figure 转换为 Pillow Image
    canvas = FigureCanvas(fig)
    canvas.draw()
    image = Image.frombytes('RGB', canvas.get_width_height(), canvas.tostring_rgb())
    frames.append(image)
    plt.close(fig)  # 关闭以节省内存

# 保存为 GIF
frames[0].save(
    gif_filename,
    save_all=True,
    append_images=frames[1:],
    duration=300,
    loop=0,
)

print(f"GIF saved successfully as '{gif_filename}'.")


#### 